# **Aspect Based Sentiment Analysis**

In [10]:
''' Import all important Libraries '''
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

# **Load Data**

In [11]:
# Dataset path
Train_path ='/Users/mahadiur/Desktop/Bongodev MLops Projects/Aspect Based Sentiment Analysis/Data/train.csv'
Test_path ='/Users/mahadiur/Desktop/Bongodev MLops Projects/Aspect Based Sentiment Analysis/Data/test.csv'

# Load by use pandas
Train_dataset = pd.read_csv(Train_path)
Test_dataset = pd.read_csv(Test_path)

# check data
print(Train_dataset.head())


                                              review   aspect sentiment
0              But the staff was so horrible to us .    staff  negative
1  To be completely fair , the only redeeming fac...     food  positive
2  The food is uniformly exceptional , with a ver...     food  positive
3  The food is uniformly exceptional , with a ver...  kitchen  positive
4  The food is uniformly exceptional , with a ver...     menu   neutral


In [18]:
''' Check aspect values in training dataset '''
Train_dataset.value_counts(['aspect'])

aspect                     
food                           340
service                        189
place                           59
prices                          56
menu                            56
                              ... 
chicken tikka marsala            1
chicken tikka                    1
chicken on rice with ginger      1
chicken in curry sauce           1
zucchini blossoms                1
Name: count, Length: 1322, dtype: int64

In [20]:
''' Check sentiment values in training dataset '''
Train_dataset.value_counts(['sentiment'])

sentiment
positive     2164
negative      807
neutral       637
Name: count, dtype: int64

In [21]:
''' Dataset information '''
Train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3608 entries, 0 to 3607
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     3608 non-null   object
 1   aspect     3608 non-null   object
 2   sentiment  3608 non-null   object
dtypes: object(3)
memory usage: 84.7+ KB


In [32]:
''' Normalize single column '''
import re
index = 0
example = Train_dataset.iloc[index]
review = example['review']
sentiment = example['sentiment']
aspect = example['aspect']
review = review.lower()
review = re.sub(r'[^a-z0-9\s]', '', review)
review = ' '.join(review.split())
review

'but the staff was so horrible to us'